<a href="https://colab.research.google.com/github/Zfeng0207/FIT3199-FYP/blob/dev%2Fdarren/Darren_NB_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path to your file
file_path = "/content/drive/MyDrive/Colab Notebooks/FIT3199-FYP/input/00_recurrent_stroke_patient.csv"

# Step 3: Load the CSV
data = pd.read_csv(file_path)

# Step 4: Preview
print(data.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   subject_id   stay_id            charttime  temperature  heartrate  \
0    10003299  31931401  2183-02-26 20:26:00         97.7       63.0   
1    10003299  31931401  2183-02-26 21:43:00          NaN       51.0   
2    10003299  31931401  2183-02-26 22:03:00          NaN       57.0   
3    10003299  31931401  2183-02-27 02:20:00          NaN       54.0   
4    10003299  31931401  2183-02-27 06:26:00         98.7       61.0   

   resprate  o2sat    sbp   dbp rhythm  ... icd_code  icd_version  \
0      15.0   97.0  146.0  65.0    NaN  ...     R918           10   
1      16.0   97.0  131.0  42.0    NaN  ...     R918           10   
2      19.0  100.0  142.0  54.0    NaN  ...     R918           10   
3      18.0   97.0  129.0  56.0    NaN  ...     R918           10   
4      15.0   98.0  123.0  53.0    NaN  ...     R918           10   

                       

In [2]:
data

,subject_id,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,...,icd_code,icd_version,icd_title,numeric_part,Stroke_Y/N,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10003299,31931401,2183-02-26 20:26:00,97.7,63.0,15.0,97.0,146.0,65.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
1,10003299,31931401,2183-02-26 21:43:00,NaN,51.0,16.0,97.0,131.0,42.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
2,10003299,31931401,2183-02-26 22:03:00,NaN,57.0,19.0,100.0,142.0,54.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
3,10003299,31931401,2183-02-27 02:20:00,NaN,54.0,18.0,97.0,129.0,56.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
4,10003299,31931401,2183-02-27 06:26:00,98.7,61.0,15.0,98.0,123.0,53.0,NaN,...,R918,10,Other nonspecific abnormal finding of lung field,918,0,F,65,2173,2008 - 2010,2183-08-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67962,19992885,35746474,2152-07-08 01:28:00,98.0,74.0,18.0,99.0,118.0,55.0,NaN,...,4423,9,LOWER EXTREMITY ANEURYSM,4423,0,M,50,2152,2014 - 2016,2152-07-24
67963,19992885,35746474,2152-07-08 01:28:00,98.0,74.0,18.0,99.0,118.0,55.0,NaN,...,E8788,9,ABN REACT-SURG PROC NEC,8788,0,M,50,2152,2014 - 2016,2152-07-24
67964,19992885,35746474,2152-07-08 03:34:00,97.8,78.0,16.0,96.0,125.0,74.0,NaN,...,9972,9,SURG COMP-PERI VASC SYST,9972,0,M,50,2152,2014 - 2016,2152-07-24
67965,19992885,35746474,2152-07-08 03:34:00,97.8,78.0,16.0,96.0,125.0,74.0,NaN,...,4423,9,LOWER EXTREMITY ANEURYSM,4423,0,M,50,2152,2014 - 2016,2152-07-24


In [3]:
import pandas as pd
import numpy as np
import gdown
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

# Step 2: Preprocessing
data['charttime'] = pd.to_datetime(data['charttime'], errors='coerce')
data = data.sort_values(by=['subject_id', 'charttime'])

columns_to_drop = ["stay_id_x", "stay_id_y", "charttime", "dod", "icd_title"]
data = data.drop(columns=columns_to_drop, errors='ignore')

numeric_data = data.select_dtypes(include=np.number)
data[numeric_data.columns] = numeric_data.fillna(numeric_data.mean())
data = data.apply(pd.to_numeric, errors='coerce')

target_column = "Stroke_Y/N"

# Step 3: Sequence generation (n_previous = 3)
def create_sequences(df, n_previous=3):
    sequences, labels = [], []
    patient_groups = df.groupby("subject_id")

    for _, group in patient_groups:
        group = group.drop(columns=["subject_id"])
        if len(group) < n_previous:
            continue

        group = group.apply(pd.to_numeric, errors='coerce')
        X_patient = group.drop(columns=[target_column]).values
        y_patient = group[target_column].values

        for i in range(len(group) - n_previous + 1):
            seq_X = X_patient[i:i + n_previous]
            seq_y = y_patient[i + n_previous - 1]
            sequences.append(seq_X)
            labels.append(seq_y)

    return np.array(sequences, dtype=np.float32), np.array(labels, dtype=np.float32)

X_seq, y_seq = create_sequences(data, n_previous=3)
X_seq = np.nan_to_num(X_seq, nan=0.0)
y_seq = np.nan_to_num(y_seq, nan=0.0)

# Step 4: Flatten sequences (for SVM)
X_flat = X_seq.reshape(X_seq.shape[0], -1)

# Step 5: Split into train, val, test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_flat, y_seq, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 6: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Step 7: Train SVM
svm_model = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Step 8: Evaluate SVM
y_pred_probs = svm_model.predict_proba(X_test_scaled)[:, 1]
y_pred_labels = (y_pred_probs >= 0.5).astype(int)

rmse = np.sqrt(np.mean((y_test - y_pred_probs) ** 2))
accuracy = accuracy_score(y_test, y_pred_labels)
auc = roc_auc_score(y_test, y_pred_probs) if len(np.unique(y_test)) > 1 else float('nan')
precision = precision_score(y_test, y_pred_labels, zero_division=0)
recall = recall_score(y_test, y_pred_labels, zero_division=0)

# Step 9: Print Metrics
print(f"\n--- SVM Evaluation ---")
print(f"RMSE: {rmse:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")



--- SVM Evaluation ---
RMSE: 0.2897
Accuracy: 0.8872
AUC: 0.8735
Precision: 0.6498
Recall: 0.2655


In [4]:
# Define 4 parameter sets to try
param_sets = [
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'},
    {'kernel': 'rbf', 'C': 10.0, 'gamma': 'scale'},
    {'kernel': 'rbf', 'C': 1.0, 'gamma': 0.01},
    {'kernel': 'linear', 'C': 1.0, 'gamma': 'auto'},  # gamma is ignored for linear but still harmless
]

# Storage for results
results = []

# Loop through parameter sets
for i, params in enumerate(param_sets, 1):
    print(f"\n🔧 Tuning Run {i}: {params}")

    # Create and train the SVM model
    model = SVC(kernel=params['kernel'], C=params['C'], gamma=params['gamma'], probability=True, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Predict on test set
    y_probs = model.predict_proba(X_test_scaled)[:, 1]
    y_preds = (y_probs >= 0.5).astype(int)

    # Evaluate
    rmse = np.sqrt(np.mean((y_test - y_probs) ** 2))
    accuracy = accuracy_score(y_test, y_preds)
    auc = roc_auc_score(y_test, y_probs) if len(np.unique(y_test)) > 1 else float('nan')
    precision = precision_score(y_test, y_preds, zero_division=0)
    recall = recall_score(y_test, y_preds, zero_division=0)

    # Log results
    results.append({
        'Run': i,
        'Kernel': params['kernel'],
        'C': params['C'],
        'Gamma': params['gamma'],
        'RMSE': rmse,
        'Accuracy': accuracy,
        'AUC': auc,
        'Precision': precision,
        'Recall': recall
    })

# Convert to DataFrame for display
results_df = pd.DataFrame(results)
print("\n📊 Comparison of Parameter Sets:")
print(results_df.to_string(index=False))



🔧 Tuning Run 1: {'kernel': 'rbf', 'C': 1.0, 'gamma': 'scale'}

🔧 Tuning Run 2: {'kernel': 'rbf', 'C': 10.0, 'gamma': 'scale'}

🔧 Tuning Run 3: {'kernel': 'rbf', 'C': 1.0, 'gamma': 0.01}

🔧 Tuning Run 4: {'kernel': 'linear', 'C': 1.0, 'gamma': 'auto'}

📊 Comparison of Parameter Sets:
 Run Kernel    C Gamma     RMSE  Accuracy      AUC  Precision   Recall
   1    rbf  1.0 scale 0.289709  0.887232 0.873544   0.649805 0.265501
   2    rbf 10.0 scale 0.277501  0.896016 0.889614   0.686916 0.350556
   3    rbf  1.0  0.01 0.297571  0.881920 0.869373   0.626238 0.201113
   4 linear  1.0  auto 0.301668  0.876302 0.846929   0.586716 0.126391


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

# === PREPROCESSING (as done before) ===
target_column = "Stroke_Y/N"

def create_sequences(df, n_previous=3):
    sequences, labels = [], []
    patient_groups = df.groupby("subject_id")

    for _, group in patient_groups:
        group = group.drop(columns=["subject_id"])
        if len(group) < n_previous:
            continue

        group = group.apply(pd.to_numeric, errors='coerce')
        X_patient = group.drop(columns=[target_column]).values
        y_patient = group[target_column].values

        for i in range(len(group) - n_previous + 1):
            seq_X = X_patient[i:i + n_previous]
            seq_y = y_patient[i + n_previous - 1]
            sequences.append(seq_X)
            labels.append(seq_y)

    return np.array(sequences, dtype=np.float32), np.array(labels, dtype=np.float32)

# Assuming `data` is already loaded and cleaned
X_seq, y_seq = create_sequences(data, n_previous=3)
X_seq = np.nan_to_num(X_seq, nan=0.0)
y_seq = np.nan_to_num(y_seq, nan=0.0)

X_flat = X_seq.reshape(X_seq.shape[0], -1)

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(X_flat, y_seq, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Prepare data variations
X_train_bin = Binarizer().fit_transform(X_train_scaled)
X_test_bin = Binarizer().transform(X_test_scaled)

X_train_nonneg = np.abs(X_train_scaled)
X_test_nonneg = np.abs(X_test_scaled)

# === DEFINE MODELS & PARAM GRIDS ===
models_and_params = [
    {
        'name': 'GaussianNB',
        'model': GaussianNB(),
        'params': {'var_smoothing': [1e-9, 1e-8, 1e-7]},
        'X_train': X_train_scaled,
        'X_test': X_test_scaled
    },
    {
        'name': 'MultinomialNB',
        'model': MultinomialNB(),
        'params': {'alpha': [0.01, 0.1, 1.0, 10.0]},
        'X_train': X_train_nonneg,
        'X_test': X_test_nonneg
    },
    {
        'name': 'BernoulliNB',
        'model': BernoulliNB(),
        'params': {
            'alpha': [0.01, 0.1, 1.0],
            'binarize': [0.0, 0.25, 0.5]
        },
        'X_train': X_train_bin,
        'X_test': X_test_bin
    },
    {
        'name': 'ComplementNB',
        'model': ComplementNB(),
        'params': {
            'alpha': [0.01, 0.1, 1.0],
            'norm': [True, False]
        },
        'X_train': X_train_nonneg,
        'X_test': X_test_nonneg
    }
]

# === RUN TUNING & EVALUATION ===
results = []

for config in models_and_params:
    print(f"\n🔍 Tuning {config['name']}...")

    grid = GridSearchCV(
        estimator=config['model'],
        param_grid=config['params'],
        scoring='recall',
        cv=3,
        n_jobs=-1
    )

    grid.fit(config['X_train'], y_train)
    best_model = grid.best_estimator_

    y_probs = best_model.predict_proba(config['X_test'])[:, 1]
    y_preds = (y_probs >= 0.5).astype(int)

    rmse = np.sqrt(np.mean((y_test - y_probs) ** 2))
    accuracy = accuracy_score(y_test, y_preds)
    auc = roc_auc_score(y_test, y_probs) if len(np.unique(y_test)) > 1 else float('nan')
    precision = precision_score(y_test, y_preds, zero_division=0)
    recall = recall_score(y_test, y_preds, zero_division=0)

    results.append({
        'Model': config['name'],
        'Best Params': grid.best_params_,
        'RMSE': rmse,
        'Accuracy': accuracy,
        'AUC': auc,
        'Precision': precision,
        'Recall': recall
    })

# === DISPLAY RESULTS ===
results_df = pd.DataFrame(results)
print("\n📊 Naive Bayes Models – Final Comparison:")
print(results_df.to_string(index=False))


KeyboardInterrupt: 

In [1]:
# Download to local Colab environment first
!wget -O mimic-iv-ecg.zip "https://physionet.org/static/published-projects/mimic-iv-ecg/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0.zip"

# Then move to Drive
!mv mimic-iv-ecg.zip /content/drive/MyDrive/


--2025-03-28 06:52:21--  https://physionet.org/static/published-projects/mimic-iv-ecg/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0.zip
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36295288053 (34G) [application/zip]
Saving to: ‘mimic-iv-ecg.zip’

mimic-iv-ecg.zip      0%[                    ]  21.77M   346KB/s    eta 27h 26m^C


In [1]:
!apt-get install -y aria2
!aria2c -x 16 -s 16 -o mimic-iv-ecg.zip "https://physionet.org/static/published-projects/mimic-iv-ecg/mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0.zip"
!mv mimic-iv-ecg.zip /content/drive/MyDrive/


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 0s (5,309 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 126209 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubun

In [2]:
!pip install awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [3]:
!aws s3 sync --no-sign-request s3://physionet-open/mimic-iv-ecg/1.0/ /content/mimic-iv-ecg/


Streaming output truncated to the last 5000 lines.
download: s3://physionet-open/mimic-iv-ecg/1.0/files/p1996/p19968468/s46368208/46368208.hea to mimic-iv-ecg/files/p1996/p19968468/s46368208/46368208.hea
download: s3://physionet-open/mimic-iv-ecg/1.0/files/p1996/p19968351/s43856348/43856348.dat to mimic-iv-ecg/files/p1996/p19968351/s43856348/43856348.dat
download: s3://physionet-open/mimic-iv-ecg/1.0/files/p1996/p19968364/s49446933/49446933.dat to mimic-iv-ecg/files/p1996/p19968364/s49446933/49446933.dat
download: s3://physionet-open/mimic-iv-ecg/1.0/files/p1996/p19968619/s44295585/44295585.dat to mimic-iv-ecg/files/p1996/p19968619/s44295585/44295585.dat
download: s3://physionet-open/mimic-iv-ecg/1.0/files/p1996/p19968656/s45328765/45328765.hea to mimic-iv-ecg/files/p1996/p19968656/s45328765/45328765.hea
download: s3://physionet-open/mimic-iv-ecg/1.0/files/p1996/p19968468/s46368208/46368208.dat to mimic-iv-ecg/files/p1996/p19968468/s46368208/46368208.dat
download: s3://physionet-open/m

In [4]:
!du -sh ./mimic-iv-ecg

99G	./mimic-iv-ecg
